# HuggingFace Model Inference Demo

This notebook demonstrates how to make parallel requests to our containerized HuggingFace model inference API.

In [38]:
import requests
import concurrent.futures
import time
import pandas as pd
from typing import List, Dict
import json

## Configuration

In [47]:
API_URL = "http://localhost:8000/analyze"
NUM_REQUESTS = 10 

sample_feedback = [
    "I'm really disappointed with this product. It's not what I expected and I feel let down by the quality.",
    "I'm absolutely delighted with this purchase! It's exactly what I needed and works perfectly!",
    "I absolutely adore this product! It's become an essential part of my daily routine and I can't imagine life without it.",
    "I'm absolutely furious about this product! It's a complete scam and I want my money back immediately!",
    "I'm really concerned about the safety of this product. The instructions are unclear and I'm worried it might be dangerous.",
    "Wow! I'm amazed by how much better this product is than I expected. It's completely transformed my experience!",
    "The product arrived on time and seems to work as described. It's okay for the price.",
    "I’m in love with this camera’s vibrant photos!",
    "I never expected this budget watch to be so durable!",
    "This unstable tripod has me worried it’ll tip over!"
    ]

## Function to make a single request

In [48]:
def make_request(texts: List[str]) -> Dict:
    """Make a single request to the API."""
    try:
        response = requests.post(
            API_URL,
            json={"texts": texts},
            timeout=30
        )
        response.raise_for_status()
        return response.json()
    except Exception as e:
        return {"error": str(e)}

## Make parallel requests

In [49]:
start_time = time.time()

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_REQUESTS) as executor:
    
    future_to_request = {
        executor.submit(make_request, [text]): text 
        for text in sample_feedback[:NUM_REQUESTS]
    }
    
    results = []
    for future in concurrent.futures.as_completed(future_to_request):
        text = future_to_request[future]
        try:
            result = future.result()
            prediction = result.get("predictions", [{}])[0]
            results.append({
                "text": text,
                "dominant_emotion": prediction.get("dominant_emotion"),
                "emotions": prediction.get("emotions", {})
            })
        except Exception as e:
            results.append({
                "text": text,
                "error": str(e)
            })

end_time = time.time()
total_time = end_time - start_time

## Display results

In [50]:
print(f"Total time for {NUM_REQUESTS} parallel requests: {total_time:.2f} seconds")
print(f"Average time per request: {total_time/NUM_REQUESTS:.2f} seconds")

df = pd.DataFrame(results)

print("\nFeedback Analysis Results:")
print("=" * 80)
for _, row in df.iterrows():
    print(f"\nFeedback: {row['text']}")
    print(f"Dominant Emotion: {row['dominant_emotion']}")
    print("Emotion Scores:")
    for emotion, score in row['emotions'].items():
        print(f"  - {emotion}: {score:.2f}")
    print("-" * 80) 

Total time for 10 parallel requests: 1.36 seconds
Average time per request: 0.14 seconds

Feedback Analysis Results:

Feedback: I'm absolutely delighted with this purchase! It's exactly what I needed and works perfectly!
Dominant Emotion: joy
Emotion Scores:
  - sadness: 0.00
  - joy: 1.00
  - love: 0.00
  - anger: 0.00
  - fear: 0.00
  - surprise: 0.00
--------------------------------------------------------------------------------

Feedback: I absolutely adore this product! It's become an essential part of my daily routine and I can't imagine life without it.
Dominant Emotion: joy
Emotion Scores:
  - sadness: 0.00
  - joy: 1.00
  - love: 0.00
  - anger: 0.00
  - fear: 0.00
  - surprise: 0.00
--------------------------------------------------------------------------------

Feedback: This unstable tripod has me worried it’ll tip over!
Dominant Emotion: fear
Emotion Scores:
  - sadness: 0.03
  - joy: 0.00
  - love: 0.00
  - anger: 0.03
  - fear: 0.94
  - surprise: 0.00
-----------------